# 00_create_dataset_no_na

The original jane-street-market-prediction dataset will be modified to suit our needs.

1. Dataset is too big (6gb), hence we need to trim it down
2. Competition is closed, we have no access to their test set. Hence we can use this train set to generate our own unseen test set
3. Need to generate our own output values (y-value) since it is indirectly provided

Follows feedback from interim presentation to remove rows with NA values

In [ ]:
''' data and math '''
import pandas as pd
import numpy as np

''' plotting images '''
from matplotlib import pyplot as plt
%matplotlib inline

''' traversing directories '''
import os
from pathlib import Path

''' utilities '''
from tqdm import tqdm

In [ ]:
''' used to reference the root directory, for directory traversal ''' 
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
mount_dir = '/content/gdrive'
root_dir = Path('/content/gdrive/My Drive/it3011_project')

Mounted at /content/gdrive


# Basic data inspection and cleaning

In [ ]:
# load data
data = pd.read_csv(root_dir/"data/jane-street-market-prediction/train.csv")
data.head()

,date,weight,resp_1,resp_2,resp_3,resp_4,resp,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,...,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100,feature_101,feature_102,feature_103,feature_104,feature_105,feature_106,feature_107,feature_108,feature_109,feature_110,feature_111,feature_112,feature_113,feature_114,feature_115,feature_116,feature_117,feature_118,feature_119,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,ts_id
0,0,0.000000,0.009916,0.014079,0.008773,0.001390,0.006270,1,-1.872746,-2.191242,-0.474163,-0.323046,0.014688,-0.002484,NaN,NaN,-0.989982,-1.055090,NaN,NaN,-2.667671,-2.001475,-1.703595,-2.196892,NaN,NaN,1.483295,1.307466,NaN,NaN,1.175200,0.967805,1.608410,1.319365,NaN,NaN,-0.515073,-0.448988,NaN,NaN,...,1.158770,NaN,3.754522,7.137163,-1.863069,NaN,0.434466,NaN,-0.292035,0.317003,-2.605820,NaN,2.896986,NaN,1.485813,4.147254,-2.238831,NaN,-0.892724,NaN,-0.156332,0.622816,-3.921523,NaN,2.561593,NaN,3.457757,6.649580,-1.472686,NaN,NaN,1.168391,8.313583,1.782433,14.018213,2.653056,12.600292,2.301488,11.445807,0
1,0,16.673515,-0.002828,-0.003226,-0.007319,-0.011114,-0.009792,-1,-1.349537,-1.704709,0.068058,0.028432,0.193794,0.138212,NaN,NaN,-0.151877,-0.384952,NaN,NaN,1.225838,0.789076,1.110580,1.102281,NaN,NaN,-0.590600,-0.625682,NaN,NaN,-0.543425,-0.547486,-0.706600,-0.667806,NaN,NaN,0.910558,0.914465,NaN,NaN,...,1.157671,NaN,1.297679,1.281956,-2.427595,NaN,0.024913,NaN,-0.413607,-0.073672,-2.434546,NaN,0.949879,NaN,0.724655,1.622137,-2.209020,NaN,-1.332492,NaN,-0.586619,-1.040491,-3.946097,NaN,0.983440,NaN,1.357907,1.612348,-1.664544,NaN,NaN,-1.178850,1.777472,-0.915458,2.831612,-1.417010,2.297459,-1.304614,1.898684,1
2,0,0.000000,0.025134,0.027607,0.033406,0.034380,0.023970,-1,0.812780,-0.256156,0.806463,0.400221,-0.614188,-0.354800,NaN,NaN,5.448261,2.668029,NaN,NaN,3.836342,2.183258,3.902698,3.045431,NaN,NaN,-1.141082,-0.979962,NaN,NaN,-1.157585,-0.966803,-1.430973,-1.103432,NaN,NaN,5.131559,4.314714,NaN,NaN,...,2.420089,NaN,0.800962,1.143663,-3.214578,NaN,1.585939,NaN,0.193996,0.953114,-2.674838,NaN,2.200085,NaN,0.537175,2.156228,-3.568648,NaN,1.193823,NaN,0.097345,0.796214,-4.090058,NaN,2.548596,NaN,0.882588,1.817895,-2.432424,NaN,NaN,6.115747,9.667908,5.542871,11.671595,7.281757,10.060014,6.638248,9.427299,2
3,0,0.000000,-0.004730,-0.003273,-0.000461,-0.000476,-0.003200,-1,1.174378,0.344640,0.066872,0.009357,-1.006373,-0.676458,NaN,NaN,4.508206,2.484260,NaN,NaN,2.902176,1.799163,3.192700,2.848359,NaN,NaN,-1.401637,-1.428248,NaN,NaN,-1.421175,-1.487976,-1.756415,-1.647543,NaN,NaN,4.766182,4.528353,NaN,NaN,...,2.330484,NaN,0.182066,1.088451,-3.527752,NaN,-1.338859,NaN,-1.257774,-1.194013,-1.719062,NaN,-0.940190,NaN,-1.510224,-1.781693,-3.373969,NaN,2.513074,NaN,0.424964,1.992887,-2.616856,NaN,0.561528,NaN,-0.994041,0.099560,-2.485993,NaN,NaN,2.838853,0.499251,3.033732,1.513488,4.397532,1.266037,3.856384,1.013469,3
4,0,0.138531,0.001252,0.002165,-0.001215,-0.006219,-0.002604,1,-3.172026,-3.093182,-0.161518,-0.128149,-0.195006,-0.143780,NaN,NaN,2.683018,1.450991,NaN,NaN,1.257761,0.632336,0.905204,0.575275,NaN,NaN,2.550883,2.484082,NaN,NaN,2.502828,2.606440,2.731251,2.566561,NaN,NaN,-1.477905,-1.722451,NaN,NaN,...,4.345282,NaN,2.737738,2.602937,-1.785502,NaN,-0.172561,NaN,-0.299516,-0.420021,-2.354611,NaN,0.762192,NaN,1.598620,0.623132,-1.742540,NaN,-0.934675,NaN,-0.373013,-1.213540,-3.677787,NaN,2.684119,NaN,2.861848,2.134804,-1.279284,NaN,NaN,0.344850,4.101145,0.614252,6.623456,0.800129,5.233243,0.362636,3.926633,4


In [ ]:
# inspect data.shape
data.shape

(2390491, 138)

In [ ]:
# some data cleaning

# drop all rows that contain NA values
data.dropna(inplace=True)

# set ts_id to index
data = data.set_index("ts_id")

# remove entries that do not contribute to learning because weight is zero
data = data[data['weight'] != 0]

# 'action' is effectively the y-value, either 1/0 representing buy/pass action. binary classification
data['action'] = ((data['resp'].values) > 0).astype(int)

data.head()

,date,weight,resp_1,resp_2,resp_3,resp_4,resp,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,...,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100,feature_101,feature_102,feature_103,feature_104,feature_105,feature_106,feature_107,feature_108,feature_109,feature_110,feature_111,feature_112,feature_113,feature_114,feature_115,feature_116,feature_117,feature_118,feature_119,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,action
ts_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
479,0,1.887768,0.011752,0.014947,0.015753,0.018192,0.020704,1,0.086238,-1.370409,-0.796239,-0.344834,-1.152042,-0.504109,-2.402996,-1.680355,0.455843,-0.815505,-0.710505,-1.066394,3.285408,1.084084,0.295333,-1.045913,1.452184,0.827972,1.109112,0.359497,3.427643,1.134909,4.224989,2.724442,1.203090,0.330962,-2.861279,-2.412291,-1.811251,-1.305059,-0.929254,-0.953782,...,-1.515613,0.791478,0.268165,-1.086886,-1.534390,-0.050836,0.185131,-0.160516,0.009224,-0.145178,-1.607949,0.380167,0.124813,0.124767,0.252838,-0.251759,-1.931394,0.216367,-1.615538,0.066251,-0.182226,-2.077377,-2.157264,0.567638,-1.422295,0.587713,0.285151,-1.812644,-1.248822,1.635075,0.759268,0.528743,-1.237860,1.610215,0.931161,2.236798,0.563850,1.023031,-0.635663,1
480,0,0.993648,0.006912,-0.000236,-0.008789,-0.021067,-0.019047,-1,2.540471,1.213258,-1.147064,-0.659254,-1.125747,-0.665765,-3.051758,-2.641164,2.721408,0.915851,2.015274,1.987354,2.551746,1.218098,1.877259,1.070328,-3.181091,-3.155554,-1.897165,-1.776704,-1.707736,-0.697636,-1.873632,-1.859861,-2.343779,-2.055000,2.165110,2.197094,2.859845,2.297837,2.502811,2.714681,...,1.347220,-0.436911,-0.006777,0.497693,-2.360903,1.958820,-1.338859,-0.684908,-1.257774,-1.194013,-0.695013,0.549498,-0.940190,-1.013225,-1.510224,-1.781693,-1.822595,2.006979,1.255436,0.030378,0.153643,0.853584,-1.242870,0.098704,-0.260217,-1.243306,-1.281847,-0.672916,-1.483448,4.213683,1.769695,3.047750,0.169149,3.513960,1.320900,3.726736,0.085196,3.322019,0.044636,0
483,0,3.942676,-0.002912,-0.000503,0.001928,0.000034,-0.000294,-1,-2.425002,-2.544622,0.369648,0.150894,-0.293128,-0.172281,0.010301,-0.203682,-0.315943,-0.891046,1.067171,0.803172,0.007300,-0.535440,-0.000169,-0.919969,-0.088526,-0.093869,-0.325506,-0.298389,-0.202820,-0.116501,-0.255703,-0.210060,-0.354502,-0.284527,1.878858,1.877591,1.300152,0.873308,1.747757,1.782523,...,3.170996,1.405887,2.592874,1.807385,-0.033861,-0.628157,-1.338859,-1.327708,-0.866745,-0.699457,-1.830639,2.616775,-0.940190,0.401246,-0.175573,-0.950078,-0.561239,-0.918237,-1.814030,-0.937312,-0.787855,-2.141688,-2.496402,2.477552,1.194468,1.196447,1.879227,0.922535,-0.214327,0.685852,4.104185,0.616460,2.895219,0.441397,3.658603,0.552314,2.694183,0.781045,2.948364,0
489,0,7.008368,-0.001950,-0.002721,-0.005041,-0.003581,-0.002728,-1,1.914959,0.138352,-0.969538,-0.444848,0.374098,0.134228,-3.042551,-2.228503,1.394126,-0.268007,0.959051,0.322820,1.361509,0.019761,1.142803,-0.193712,-3.718467,-3.085715,-1.471414,-1.060732,-1.365612,-0.411624,-1.488710,-1.063169,-1.841449,-1.202531,1.140633,0.801082,2.431071,1.425917,2.127173,1.755724,...,1.794686,0.409957,0.350763,0.809837,-0.695523,0.635341,-0.293719,-0.520128,-0.304748,0.028573,-1.155759,1.556029,-0.305690,-0.084751,-0.133821,0.227583,-1.018472,0.636929,-0.514197,-0.261071,-0.375442,-0.369468,-1.518443,1.099684,0.315145,0.167509,0.035635,0.528757,-0.631493,1.074537,1.351757,-0.134683,-1.304993,0.929085,1.227290,1.183403,0.628978,0.621575,-0.071838,0
491,0,0.319410,0.013140,0.018152,0.028973,0.018599,0.007430

In [ ]:
# inspect data.shape after data cleaning
data.shape

(1664520, 138)

In [ ]:
# just to have an understanding of the number of positive vs negative samples
data['action'].sum()/data.shape[0]

0.5047022565063802

In [ ]:
# inspect data.date
data.date.value_counts()

44     11697
459    10176
45      8655
85      8181
47      8161
       ...  
14      1170
270      450
36       195
2         22
294       11
Name: date, Length: 500, dtype: int64

We have about 1.98 million rows in total

We can see that there are 500 different dates, and the description of the data on the official kaggle site indicates that these dates do not have any relationship with each other, but simply used to group entries and calculate an overall evaluation score. Therefore we can explore a subset of this data, which is grouped by their dates in order to reduce the size of the dataset (6gb is too big) and create our own test set as well.

In [ ]:
data_date_keys = data.date.value_counts().keys()
data_date_keys

Int64Index([ 44, 459,  45,  85,  47,  38,  12,  43, 161,  18,
            ...
            171, 122, 102, 136, 113,  14, 270,  36,   2, 294],
           dtype='int64', length=500)

In [ ]:
data_date_values = data.date.value_counts().values
data_date_values

array([11697, 10176,  8655,  8181,  8161,  8152,  7786,  7388,  7350,
        7310,  6965,  6884,  6740,  6689,  6561,  6402,  6319,  6296,
        6242,  6086,  6081,  5982,  5855,  5855,  5735,  5718,  5674,
        5625,  5435,  5430,  5386,  5373,  5341,  5328,  5244,  5214,
        5112,  5092,  5078,  5055,  5004,  4993,  4861,  4820,  4817,
        4813,  4802,  4768,  4765,  4748,  4747,  4723,  4686,  4670,
        4657,  4649,  4637,  4630,  4624,  4593,  4592,  4570,  4531,
        4484,  4481,  4473,  4468,  4467,  4397,  4371,  4341,  4311,
        4295,  4287,  4271,  4271,  4257,  4256,  4254,  4231,  4229,
        4218,  4201,  4199,  4193,  4188,  4180,  4172,  4152,  4103,
        4067,  4042,  4029,  4021,  4016,  4014,  4013,  4006,  4005,
        3970,  3949,  3939,  3938,  3883,  3883,  3880,  3857,  3855,
        3848,  3841,  3835,  3834,  3822,  3815,  3792,  3775,  3761,
        3760,  3747,  3741,  3738,  3729,  3729,  3728,  3724,  3706,
        3704,  3698,

# Getting the slice of data that we want

In [ ]:
keys = []
key_counts = []
percentage = []
sum_percentage = []
for i, key in enumerate(data_date_keys):
  keys.append(key)
  key_counts.append(data_date_values[i])
  percentage.append(data_date_values[i]/data.shape[0])
  sum_percentage.append(sum(data_date_values[:i+1])/data.shape[0])

In [ ]:
data_key_value = pd.DataFrame(list(zip(keys, key_counts, percentage, sum_percentage)), 
               columns =['keys', 'key_counts', 'percentage', 'sum_percentage']) 
data_key_value

,keys,key_counts,percentage,sum_percentage
0,44,11697,0.007027,0.007027
1,459,10176,0.006113,0.013141
2,45,8655,0.005200,0.018340
3,85,8181,0.004915,0.023255
4,47,8161,0.004903,0.028158
...,...,...,...,...
495,14,1170,0.000703,0.999593
496,270,450,0.000270,0.999863
497,36,195,0.000117,0.999980
498,2,22,0.000013,0.999993


We want to reduce our 6gb total dataset to the following: approx 1gb total, with 700mb train + 300mb test. We will get the appropriate slice of data using the dataframe above

In [ ]:
nrows_original = 2390491
nrows_cleaned = data.shape[0]

# approximate size in megabytes 
size_cleaned = nrows_cleaned / nrows_original * 6000
size_cleaned

4177.8530017473395

In [ ]:
index_700mb = np.min(np.where(data_key_value.sum_percentage > 700/size_cleaned))
print(index_700mb)

index_1000mb = np.min(np.where(data_key_value.sum_percentage > 1000/size_cleaned))
print(index_1000mb)

43
69


In [ ]:
# Therefore, we will slice the keys ordered by their count in descending order as such
# 1. train: [:index_700mb+1]
# 2. test: [index_700mb+1:index_1000mb+1]

train = data[data.date.isin(data_date_keys[0:index_700mb+1])]
print(train.shape)

test = data[data.date.isin(data_date_keys[index_700mb+1:index_1000mb+1])]
print(test.shape)

# verify that test is about 30% of the new dataset
test.shape[0] / (test.shape[0] + train.shape[0])

(279331, 138)
(120163, 138)


0.3007879968159722

In [ ]:
# index_70_percent = np.min(np.where(data_key_value.sum_percentage > 0.7))
# print(index_70_percent)

In [ ]:
# # Therefore, we will slice the keys ordered by their count in descending order as such
# # 1. train: [:index_70_percent+1]
# # 2. test: [index_70_percent+1:]

# train = data[data.date.isin(data_date_keys[:index_70_percent+1])]
# print(train.shape)

# test = data[data.date.isin(data_date_keys[index_70_percent+1:])]
# print(test.shape)

# # verify that test is about 30% of the new dataset
# test.shape[0] / (test.shape[0] + train.shape[0])

# Saving this slice in a separate csv

In [ ]:
train.to_csv(root_dir/"data/train_no_na.csv", index = False)
test.to_csv(root_dir/"data/test_no_na.csv", index = False)
print("saved")

saved


In [ ]:
# open up to check
train_check = pd.read_csv(root_dir/"data/train_no_na.csv")
train_check.head()

,date,weight,resp_1,resp_2,resp_3,resp_4,resp,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,...,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100,feature_101,feature_102,feature_103,feature_104,feature_105,feature_106,feature_107,feature_108,feature_109,feature_110,feature_111,feature_112,feature_113,feature_114,feature_115,feature_116,feature_117,feature_118,feature_119,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,action
0,1,0.049702,-0.004451,-0.007539,-0.017379,-0.029313,-0.023142,1,-3.172026,-3.093182,1.512675,0.779714,1.517770,0.837579,3.318302,2.484226,-1.564522,-1.441416,1.540976,1.267065,3.524811,1.747983,0.241033,-0.772710,3.958276,3.476400,0.713982,0.275969,4.338747,2.011426,4.175938,3.415048,1.762400,0.977785,-0.543780,-0.594601,-2.511533,-2.373553,-1.074734,-1.394782,...,-1.515613,5.677752,10.799034,19.120732,0.606195,0.420156,0.010075,0.501981,1.128881,0.041071,-1.400465,5.846829,1.819568,4.581050,9.782705,3.609269,0.446851,0.671924,-1.977097,0.402852,1.093017,1.627502,-1.976948,5.589030,0.891432,6.175594,12.310337,13.717248,0.393055,1.254486,3.425070,0.687301,1.372581,0.777496,2.636138,0.676131,1.282392,0.335632,0.629061,0
1,1,4.728038,0.002833,0.011364,0.048555,0.088977,0.065740,-1,0.377763,-0.720159,-1.228130,-0.687216,0.160308,0.063995,-2.889253,-2.438545,-1.606810,-1.440496,-5.706177,-5.818397,-4.891262,-3.296428,-2.871597,-3.087057,-4.517617,-4.224645,-0.789653,-0.597041,-8.344222,-4.579602,-4.825100,-4.663691,-3.086572,-2.747776,0.189471,0.125526,0.982250,0.571303,0.381801,0.243608,...,2.307272,10.663898,8.128268,1.683005,3.714004,11.439082,6.123848,14.691253,12.129276,9.022861,2.029396,17.532307,10.318108,20.456292,20.298860,18.538467,3.030780,13.707741,4.712793,10.552746,9.029145,7.431195,2.871210,18.425188,8.398458,17.649093,16.917675,12.448589,2.754040,-0.025439,-0.366213,-0.445202,-1.287065,-0.092480,-0.748794,-0.107008,-0.543547,-0.095256,-0.495098,1
2,1,0.268049,-0.017070,-0.022214,-0.054411,-0.077741,-0.054330,-1,-1.177102,-1.734608,-0.689703,-0.455382,-0.206507,-0.145776,-1.621706,-1.558800,-3.537086,-2.195517,-1.490487,-2.100038,-2.005773,-1.515795,-1.453697,-1.994833,-1.920424,-2.193636,-1.432379,-1.514575,-4.014524,-2.235931,-1.507583,-1.657576,-1.865603,-1.830719,0.332970,0.292664,0.335046,0.204754,0.470814,0.418177,...,0.137237,2.309809,1.960359,0.776142,0.014420,0.947441,0.731767,0.101166,0.033244,0.408090,-1.104629,5.410173,3.340855,2.016045,2.061264,3.803088,0.438459,1.065228,-0.610402,0.020834,-0.195666,-0.556417,-1.707052,4.392697,2.581839,2.767720,2.677004,2.626261,0.175910,-0.635002,0.689418,-0.966584,-0.289771,-0.607015,0.500926,-1.020716,0.141768,-1.000138,-0.101238,0
3,1,0.564904,0.007865,0.000772,0.008849,0.045942,0.025119,1,6.127985,2.936178,3.082458,1.505542,1.750326,0.787248,6.689631,4.168929,3.068928,0.490253,-1.110881,-1.372875,4.540641,1.875119,1.511360,0.024486,7.528713,5.271071,3.675274,2.077969,4.099027,1.487177,5.105936,3.464596,3.874481,2.139252,-0.929582,-0.863768,-1.230530,-0.768799,-1.752233,-1.956966,...,0.859138,1.626593,0.108763,0.939902,-0.350516,2.506351,1.388905,1.046260,0.664752,1.061141,-0.502214,4.750734,2.143300,1.462090,1.458714,2.686717,0.098941,2.778227,0.505230,0.734830,0.167722,0.730993,-0.899488,3.731839,1.843139,2.016198,1.257882,2.012624,-0.085523,2.197569,0.964484,1.529813,-0.244116,1.675068,0.323785,2.365470,0.152602,2.154636,0.141610,1
4,1,3.077178,0.004767,0.008116,0.004115,-0.010378,-0.004408,-1,-0.460502,-1.415140,0.188366,0.054229,0.154061,0.061002,-0.458767,-0.485132,-0.448810,-0.983954,-1.491174,-1.899205,-3

In [ ]:
train_check.shape

(279331, 138)

In [ ]:
# open up to check
test_check = pd.read_csv(root_dir/"data/test_no_na.csv")
test_check.head()

,date,weight,resp_1,resp_2,resp_3,resp_4,resp,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,...,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100,feature_101,feature_102,feature_103,feature_104,feature_105,feature_106,feature_107,feature_108,feature_109,feature_110,feature_111,feature_112,feature_113,feature_114,feature_115,feature_116,feature_117,feature_118,feature_119,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,action
0,8,0.440809,-0.000022,0.001512,-0.007352,-0.013615,-0.005587,1,-3.172026,-3.093182,1.640494,1.043663,0.551173,0.319730,2.755769,2.396369,0.951803,0.010342,2.562343,2.881376,0.510994,-0.121935,0.633409,-0.033665,3.682967,3.749346,1.403444,1.049410,5.075588,2.880323,2.219282,1.940506,1.522328,1.045326,0.210625,0.162829,1.589514,1.309203,0.424108,0.330603,...,-1.515613,-2.115890,-1.746285,-1.086886,-5.168435,1.425700,-1.338859,3.110613,-1.257774,-1.194013,-0.949151,-0.216645,-0.940190,0.671226,-1.510224,-1.781693,-2.332853,0.772851,-3.548454,1.316682,-1.838865,-3.807339,-1.925905,-0.936224,-2.922627,0.022886,-4.159270,-3.202494,-2.063330,7.538535,3.162755,5.403643,0.939673,5.428928,1.847902,7.131680,1.357367,6.195365,1.019040,0
1,8,0.545093,-0.000272,-0.000543,-0.002930,-0.007432,-0.006172,-1,2.082171,0.685516,0.226639,0.069729,0.477435,0.225188,-1.677206,-1.381027,2.795832,0.833084,-1.320173,-1.697653,1.147320,0.117908,1.930971,0.971613,-2.037961,-1.981020,-1.484275,-1.277124,-2.659810,-1.158940,-1.500840,-1.316655,-1.857011,-1.465793,0.612503,0.445456,2.917541,2.204255,1.174154,0.992190,...,0.875003,-1.013578,-0.419133,0.221976,-3.104776,0.181095,-1.338859,0.218856,-0.474671,-1.194013,-1.345103,-1.313430,-0.940190,-0.644461,-1.021404,-1.781693,-2.798531,-0.232274,0.929275,-0.021096,0.002392,0.549046,-2.084473,-1.461678,-0.715393,-1.441384,-1.726469,-1.114429,-2.109171,2.455170,0.794366,3.136098,1.049278,2.550544,1.206773,3.664555,0.940459,3.418088,0.915626,0
2,8,1.303981,-0.017345,-0.025392,-0.085639,-0.167402,-0.141614,1,-0.077279,-1.820151,-2.334332,-0.852412,-2.087617,-0.753239,-3.850351,-2.166498,-2.425161,-1.678809,-4.147471,-3.230987,-5.572548,-2.744598,-1.762161,-2.153982,0.133479,-0.027758,0.748673,0.041054,0.405372,-0.011052,0.556048,0.004902,0.806105,0.028085,-4.505502,-2.893077,-0.712518,-0.263718,-4.747880,-3.624175,...,2.233006,7.371675,8.605624,4.015008,1.241583,0.299784,-0.306259,-0.278870,-0.300277,-0.008652,-1.447166,6.457834,0.678203,2.890113,2.481619,4.042433,0.650636,0.677731,-1.972311,0.202421,0.113197,-0.739799,-1.924646,6.777267,1.369930,6.206831,6.967825,4.641536,0.729669,1.187223,-1.948327,0.803703,-1.868713,0.895265,-2.462197,1.002079,-2.036157,0.887088,-1.803716,0
3,8,0.623796,0.019881,0.026444,0.036770,0.039449,0.040266,-1,-2.069387,-2.384638,0.093850,0.014602,0.598461,0.323845,-0.611983,-0.593488,-1.021256,-1.198279,-0.506991,-0.930389,-0.484092,-0.760608,-0.585610,-1.422149,-1.095715,-1.153584,-0.391184,-0.333999,-0.505059,-0.185788,-0.323510,-0.240809,-0.441369,-0.323176,1.939341,2.021468,0.744131,0.441077,0.806306,0.675441,...,0.776746,0.950702,0.588552,0.434173,-0.069814,1.931377,-1.338859,-0.030189,-0.321006,-0.322268,-0.683069,4.304920,-0.940190,0.511520,-0.003383,-0.377037,0.029002,2.361998,-1.488002,0.026090,-0.377904,-1.167152,-1.007667,3.637853,-0.817756,0.954954,0.330764,-0.204535,-0.021068,0.207892,3.241817,-0.555941,0.462985,0.023717,2.770049,-0.263278,1.482547,-0.409991,0.880233,1
4,8,0.214217,-0.016728,-0.024640,-0.074676,-0.173864,-0.125129,1,-3.172026,-3.093182,-2.212657,-0.799346,-2.012364,-0.720457,-3.122134,-1.776318,-1.016846

In [ ]:
test_check.shape

(120163, 138)

# Check the maximum scores attainable

In [ ]:
# create train/test sets
features = [feature for feature in test_check.keys() if "feature" in feature]
x_train = train_check.loc[:, features].values
y_train = train_check.loc[:,['action']].values.flatten()
x_test = test_check.loc[:, features].values
y_test = test_check.loc[:,['action']].values.flatten()
print("train/test set created")

In [ ]:
# create the utility score, which takes in the prediction value and the ground truth action and generates a score
# link: https://www.kaggle.com/c/jane-street-market-prediction/overview/evaluation

# data: original train/test data    action: the y-value. can either be y_pred or original values too, if we want the max score attainable
def utility_score(data, action): 
  dates_set = set(data.date.values)
  dates = data.loc[:, ['date']].values.flatten()
  weights = data.loc[:, ['weight']].values.flatten()
  resps = data.loc[:, ['resp']].values.flatten()
  actions = action.flatten()

  i = len(dates_set)
  p_i = []

  for date in dates_set:
    indices = np.where(dates == date)[0]
    p_i_temp = 0
    for j in indices:
      p_i_temp = p_i_temp + weights[j] * resps[j] * actions[j]
    p_i.append(p_i_temp)
  
  p_i_squared = [p_i1*p_i2 for p_i1,p_i2 in zip(p_i,p_i)]
  t = ( sum(p_i) / np.sqrt(sum(p_i_squared)) ) * np.sqrt(250/i)
  u = min(max(t, 0), 6) * sum(p_i)

  return u

def max_train_utility_score(data, action):
  return utility_score(data, action)

def max_test_utility_score(data, action):
  return utility_score(data, action)

In [ ]:
# maximum train utility score, times 0.1 to account for 10-fold cross validation
max_train_utility_score(train_check, y_train) * 0.1

3733.3938668944825

In [ ]:
# maximum test utility score
max_test_utility_score(test_check, y_test) 

15301.724057177264

# All looks good, we are ready to use this train/test set for machine learning